In [1]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr
import os

from langchain_community.document_loaders import UnstructuredExcelLoader

In [2]:
path=os.path.dirname(os.getcwd())
path=os.path.join(path,'Datasets/Test.xlsx')

loader=UnstructuredExcelLoader(path,mode="elements")
docs=loader.load() 

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

In [4]:
from langchain_openai import OpenAIEmbeddings
embedding=OpenAIEmbeddings()


In [5]:

text_split=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)
docs=text_split.split_documents(docs)

In [6]:
from langchain_community.vectorstores.utils import filter_complex_metadata
docs=filter_complex_metadata(docs)

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7aa3f8ad38d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7aa3f8ac5990>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None)

In [18]:
from langchain_core.retrievers import BaseRetriever
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import filter_complex_metadata



def config_retrivalChain(docs):
    """Read Docs, configure retriival and the chain"""
    docs=filter_complex_metadata(docs)
    text_split=RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=False
    )
    embedding=OpenAIEmbeddings()
    docs=text_split.split_documents(docs)
    vectorStore=DocArrayInMemorySearch(docs,embedding)
    return vectorStore.as_retriever(search_type='mmr',search_kwargs={'k':4,"fetch_k":4})




In [19]:
from langchain.chains.base import Chain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import UnstructuredExcelLoader

path=os.path.dirname(os.getcwd())
path=os.path.join(path,'Datasets/Test.xlsx')

loader=UnstructuredExcelLoader(path,mode="elements")
Doc=loader.load()

def config_chain(retriver:BaseRetriever)->Chain:
    """Configure Chain with A retriver"""
    memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)
    LLM=ChatOpenAI(temperature=0.3, model='gpt-3.5-turbo-1106',streaming=True)
    return ConversationalRetrievalChain.from_llm(
        LLM, retriver, memory=memory,verbose=True,max_tokens_limit=4000
    )

def chatConveration(docs=Doc):
    retriver=config_retrivalChain(docs)
    chain=config_chain(retriver)
    return chain



In [21]:
import gradio as gr

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    chain=chatConveration()
    response=chain.run(input=message,history=history_langchain_format)
    return response.content


gr.ChatInterface(predict).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


/home/deon/Project/SakuraSky/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Traceback (most recent call last):
  File "/home/deon/Project/SakuraSky/env/lib/python3.11/site-packages/gradio/queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/deon/Project/SakuraSky/env/lib/python3.11/site-packages/gradio/route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/deon/Project/SakuraSky/env/lib/python3.11/site-packages/gradio/blocks.py", line 1847, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/deon/Project/SakuraSky/env/lib/python3.11/site-packages/gr